In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('top10K-TMDB-movies.csv')
df.head()

,id,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count
0,278,The Shawshank Redemption,"Drama,Crime",en,Framed in the 1940s for the double murder of h...,94.075,1994-09-23,8.7,21862
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance",hi,"Raj is a rich, carefree, happy-go-lucky second...",25.408,1995-10-19,8.7,3731
2,238,The Godfather,"Drama,Crime",en,"Spanning the years 1945 to 1955, a chronicle o...",90.585,1972-03-14,8.7,16280
3,424,Schindler's List,"Drama,History,War",en,The true story of how businessman Oskar Schind...,44.761,1993-12-15,8.6,12959
4,240,The Godfather: Part II,"Drama,Crime",en,In the continuing saga of the Corleone crime f...,57.749,1974-12-20,8.6,9811


In [3]:
df.describe()

,id,popularity,vote_average,vote_count
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,161243.505000,34.697267,6.621150,1547.309400
std,211422.046043,211.684175,0.766231,2648.295789
min,5.000000,0.600000,4.600000,200.000000
25%,10127.750000,9.154750,6.100000,315.000000
50%,30002.500000,13.637500,6.600000,583.500000
75%,310133.500000,25.651250,7.200000,1460.000000
max,934761.000000,10436.917000,8.700000,31917.000000


In [4]:
df.shape

(10000, 9)

In [5]:
df.columns

Index(['id', 'title', 'genre', 'original_language', 'overview', 'popularity',
       'release_date', 'vote_average', 'vote_count'],
      dtype='object')

In [6]:
df.isnull().sum()

id                    0
title                 0
genre                 3
original_language     0
overview             13
popularity            0
release_date          0
vote_average          0
vote_count            0
dtype: int64

In [7]:
df = df.dropna().reset_index(inplace=False).drop(['index'], axis = 1)

In [8]:
df.isnull().sum()

id                   0
title                0
genre                0
original_language    0
overview             0
popularity           0
release_date         0
vote_average         0
vote_count           0
dtype: int64

In [10]:
df = df[['id', 'title', 'overview', 'original_language', 'genre']]

In [12]:
data = pd.DataFrame()
data['tags'] = df['overview']+df['genre']+df['original_language']
data['id'] = df['id']
data['title'] = df['title']
data

,tags,id,title
0,Framed in the 1940s for the double murder of h...,278,The Shawshank Redemption
1,"Raj is a rich, carefree, happy-go-lucky second...",19404,Dilwale Dulhania Le Jayenge
2,"Spanning the years 1945 to 1955, a chronicle o...",238,The Godfather
3,The true story of how businessman Oskar Schind...,424,Schindler's List
4,In the continuing saga of the Corleone crime f...,240,The Godfather: Part II
...,...,...,...
9980,"The story follows the adventures of Aang, a yo...",10196,The Last Airbender
9981,The sharks take bite out of the East Coast whe...,331446,Sharknado 3: Oh Hell No!
9982,"During World War II, a brave, patriotic Americ...",13995,Captain America
9983,A man named Farmer sets out to rescue his kidn...,2312,In the Name of the King: A Dungeon Siege Tale


In [13]:
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
cv = CountVectorizer(max_features=len(data), stop_words='english')

In [17]:
cv

CountVectorizer(max_features=9985, stop_words='english')

In [18]:
vectors = cv.fit_transform(data['tags'].values.astype('U')).toarray()

In [20]:
vectors.shape

(9985, 9985)

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
similarity = cosine_similarity(vectors)
similarity.shape

(9985, 9985)

In [23]:
data[data['title']=='The Godfather']

,tags,id,title
2,"Spanning the years 1945 to 1955, a chronicle o...",238,The Godfather


In [24]:
data[data['title']=='The Godfather'].index[0]

2

In [27]:
def recommend(movie):
    index=data[data['title']==movie].index[0]
    distance = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda vector:vector[1])
    recommend_movie=[]
    for i in distance[1:6]:
        movies_id=data.iloc[i[0]].id
        recommend_movie.append(data.iloc[i[0]].title)
    return recommend_movie

In [28]:
recommend('The Godfather')

['The Godfather: Part II', 'Felon', 'House of Gucci', 'The Outsider', 'Gotti']

In [32]:
import pickle

In [34]:
pickle.dump(data,open('movies_list.pkl','wb'))

In [35]:
pickle.dump(similarity,open('similarity.pkl','wb'))

In [36]:
pickle.load(open('movies_list.pkl', 'rb'))

,tags,id,title
0,Framed in the 1940s for the double murder of h...,278,The Shawshank Redemption
1,"Raj is a rich, carefree, happy-go-lucky second...",19404,Dilwale Dulhania Le Jayenge
2,"Spanning the years 1945 to 1955, a chronicle o...",238,The Godfather
3,The true story of how businessman Oskar Schind...,424,Schindler's List
4,In the continuing saga of the Corleone crime f...,240,The Godfather: Part II
...,...,...,...
9980,"The story follows the adventures of Aang, a yo...",10196,The Last Airbender
9981,The sharks take bite out of the East Coast whe...,331446,Sharknado 3: Oh Hell No!
9982,"During World War II, a brave, patriotic Americ...",13995,Captain America
9983,A man named Farmer sets out to rescue his kidn...,2312,In the Name of the King: A Dungeon Siege Tale
